<a href="https://colab.research.google.com/github/ramari96/Algoritmos_de_optimizacion_2026/blob/main/Seminario_Algoritmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos: Emma Otiavén Carracedo - Ramiro Rivas Fernández  <br>
Url: https://github.com/.../03MAIR---Algoritmos-de-Optimizacion---2019/tree/master/SEMINARIO<br>
Problema:
> 1. Sesiones de doblaje <br>
>2. Organizar los horarios de partidos de La Liga<br>
>3. Combinar cifras y operaciones

Descripción del problema: Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las
tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de
doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de
grabación independientemente del número de tomas que se graben. No es posible grabar más
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible.

(*) La respuesta es obligatoria





                                        

#1.a ¿Cuántas posibilidades hay sin tener en cuenta las restricciones?

### Posibilidades sin tener en cuenta las restricciones

En el problema se nos pide planificar las sesiones por día de manera que minimicemos el gasto de los actores. Si tratamos al problema como un n-upla de 30 elementos (tomas) obtendríamos:

$$T = (T_1,...,T_{30})$$

¿Cuántas formas de ordenar esta n-upla existen?

Sin tener en cuenta las restricciones del problema, las posibilidades se pueden calcular como permutaciones en las que **no se pueden dar repeticiones** y en las que el orden **importa**.

Siguiendo la fórmula de permutación sin repetición y donde el orden importa: $$P_n = n!$$

Para este problema en concreto: $P_{30} = 30!$

#1.b ¿Cuántas posibilidades teniendo en cuenta las restricciones?
### Posibilidades teniendo en cuenta las restricciones

La restricción que se nos plantea es que como máximo se pueden grabar 6 tomas por día, entonces la pregunta pasa a ser: ¿cuántos conjuntos de 6 elementos puedo formar con los elementos de una lista de 30 elementos?

Esto serían combinaciones de 6 elementos entre 30 donde no se puede repetir e importa el orden en el que están dichos elementos:

$$C\binom{n}{k} = \frac{n!}{k!(n-k)!}$$

Para nuestro caso: $C\binom{30}{6} = \frac{30!}{6! 24!} = 593775$




#2. ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, argumentalo)

Al inicio planteamos una primera solución basada en una estructura óptima formada por una lista de días, donde cada día contiene una lista de tomas, y cada toma incluye el conjunto de actores que deben coincidir. Esta representación permite modelar directamente la planificación (día → tomas → actores), facilita la comprobación de restricciones como el máximo de seis tomas por día y la coincidencia de actores, y además es flexible para generar, evaluar y optimizar soluciones dentro de un espacio combinatorio.
El ejemplo de esta propuesta era:
- Día 1: tomas con actores {A, B}, luego {C}, luego {A, D}.
- Día 2: tomas con actores {B, C} y luego {A, C, D}.

Posteriormente planteamos una segunda alternativa: modelar las tomas como un arreglo unidimensional de treinta elementos, donde cada elemento representa una toma. Este enfoque reduce la cantidad de operaciones necesarias porque la matriz de organización del rodaje ya está fijada y no forma parte del problema a resolver. Solo se utiliza como referencia para calcular los costes de la función objetivo, pero no interviene en la estructura de la solución.

La estructura de datos sera por tanto un arreglo del tipo:

$$T = [1,2,3,...,30]$$ donde cada $T_i$ representa el número de una de las tomas.


#3. :¿Cuál es la función objetivo? ¿Es un problema de maximización o minimización?

Para nuestro problema, debemos tener en cuenta que elementos aportan datos para la solución, haciendo un análisis de la letra obtenemos:

- 30 tomas que debemos grabar.
- 10 actores que participan en las tomas, cada uno aparecerá en la matriz de organización como 1 si aparece en la toma, o 0 en caso contrario.
- Los actores cobran por día y no por toma, esto quiere decir que si un actor aparece en todas las tomas de un día cobrará igual que si aparece en sólo 1.

Con esto en mente, lo que queremos es calcular cuánto nos costará el rodaje, sabiendo que sólo podemos grabar un máximo de 6 tomas por día.

$$F = \sum_{i=1}^{n} CosteDia_i$$

Es un problema de *minimización* ya que lo que queremos es minimizar el coste por día de rodaje.







#4. Diseña un algoritmo para resolver el problema por fuerza bruta

In [33]:
import pandas as pd
import numpy as np
import itertools

###################################################################################################################
################################## PARTE 1: TRATADO DE DATOS DEL PROBLEMA #########################################
###################################################################################################################

#Lectura de datos y cargado del dataframe que contiene la matriz de organización del rodaje
data_url = "https://docs.google.com/spreadsheets/d/1Ipn6IrbQP4ax8zOnivdBIw2lN0JISkJG4fXndYd27U0/export?format=csv"
matriz_de_rodaje = pd.read_csv(data_url)

#Transformación de los encabezados de las columnas a Actor 1, ..., Actor10
matriz_de_rodaje = matriz_de_rodaje.rename(columns={
    'Actor': 'Actor 1',
    'Unnamed: 2': 'Actor 2',
    'Unnamed: 3': 'Actor 3',
    'Unnamed: 4': 'Actor 4',
    'Unnamed: 5': 'Actor 5',
    'Unnamed: 6': 'Actor 6',
    'Unnamed: 7': 'Actor 7',
    'Unnamed: 8': 'Actor 8',
    'Unnamed: 9': 'Actor 9',
    'Unnamed: 10': 'Actor 10',
})

#Procesado de las tomas para convertirlas en conjuntos de actores unicos (así podemos saber el coste de cada toma)
tomas = matriz_de_rodaje.columns[1:]
tomas_actores = []

#Limpieza del dataframe de aquellas filas que no aportan lo que necesitamos
matriz_de_rodaje = matriz_de_rodaje.drop(index=0).drop(index=[31,32])

for _, row in matriz_de_rodaje.iterrows():
  actores_unicos = set(actores for actores in tomas if row[actores]==1) #Se busca que row[actores]==1 porque eso quiere decir que el actor participa en la toma
  tomas_actores.append(actores_unicos)


###################################################################################################################
################################## PARTE 2: FUNCIÓN DE COSTE PARA EL PROBLEMA #####################################
###################################################################################################################

def coste_total(tomas):
  #Cálculo del coste
  coste = 0

  #Se recorre la solución actual de 6 en 6 (esto es por la restricción de los días)
  for i in range(0, len(tomas), 6):

    #Vemos que tomas tocan en el día actual (selecciona las tomas de 6 en 6 en el orden que aparecen)
    bloque_actual = tomas[i:i+6]

    actores_del_bloque = set()

    for toma in bloque_actual:
      actores_de_la_toma = tomas_actores[toma - 1] #Porque las tomas van de 1-30 y los indexes de 0-29
      actores_del_bloque.update(actores_de_la_toma)

    coste += len(actores_del_bloque) #Esto es porque se sabe que los actores cobran 1 por día
  return coste

###################################################################################################################
################################## PARTE 3: RESOLUCIÓN POR FUERZA BRUTA ###########################################
###################################################################################################################

def fuerza_bruta(numero_tomas):
  '''
  Resuelve el problema para un número limitado de tomas, esto es porque de abarcar las 30 tomas tendríamos demasiadas
  combinaciones posibles
  '''

  #Creo la solución inicial T para el problema con la cantidad de tomas indicada
  S = [int(s) for s in np.random.permutation(np.arange(1,numero_tomas+1))]

  #inicializo mejor_coste y mejor_solucion
  mejor_coste = float('inf')
  mejor_solucion = []

  for permutacion in itertools.permutations(S):
    #defino la solucion actual y el coste actual como S y el coste implicado en S
    solucion_actual = permutacion
    coste_actual = coste_total(permutacion)

    #si el coste calculado es mejor que el mejor guardado, lo cambio
    if coste_actual < mejor_coste:
      mejor_coste = coste_actual
      mejor_solucion = solucion_actual

  return mejor_solucion, mejor_coste


###################################################################################################################
################################## PARTE 3: RESOLUCIÓN POR FUERZA BRUTA ###########################################
###################################################################################################################

solucion, coste = fuerza_bruta(8)
print(f'La solución es {solucion} y el coste asociado es {coste}')
print(tomas_actores)




La solución es (6, 2, 5, 7, 4, 1, 3, 8) y el coste asociado es 12
[{'Actor 4', 'Actor 1', 'Actor 5', 'Actor 2', 'Actor 3'}, {'Actor 5', 'Actor 4', 'Actor 3'}, {'Actor 2', 'Actor 7', 'Actor 5'}, {'Actor 7', 'Actor 2', 'Actor 8', 'Actor 1'}, {'Actor 2', 'Actor 4', 'Actor 8'}, {'Actor 2', 'Actor 4', 'Actor 1', 'Actor 5'}, {'Actor 2', 'Actor 4', 'Actor 1', 'Actor 5'}, {'Actor 2', 'Actor 1', 'Actor 6'}, {'Actor 2', 'Actor 4', 'Actor 1'}, {'Actor 2', 'Actor 9', 'Actor 1', 'Actor 6'}, {'Actor 1', 'Actor 5', 'Actor 2', 'Actor 8', 'Actor 3'}, {'Actor 4', 'Actor 1', 'Actor 6', 'Actor 2', 'Actor 3'}, {'Actor 4', 'Actor 1', 'Actor 5'}, {'Actor 6', 'Actor 1', 'Actor 3'}, {'Actor 7', 'Actor 2', 'Actor 1'}, {'Actor 4', 'Actor 10'}, {'Actor 1', 'Actor 3'}, {'Actor 6', 'Actor 3'}, {'Actor 1', 'Actor 3'}, {'Actor 5', 'Actor 4', 'Actor 1', 'Actor 3'}, {'Actor 6', 'Actor 8'}, {'Actor 2', 'Actor 4', 'Actor 1', 'Actor 3'}, {'Actor 1', 'Actor 3'}, {'Actor 6', 'Actor 3'}, {'Actor 2', 'Actor 4', 'Actor 10', 'A

#5. Calcula la complejidad del algoritmo por fuerza bruta

Para el cálculo de la complejidad del algoritmo *fuerza_bruta* debemos considerar:
1. *for de itertools:* calcula todas las permutaciones posibles sin repetición para un conjunto de n elementos, por lo que sabemos que su complejidad algorítmica es $O(n!)$
2. *coste de valores*: al calcular el coste de la lista de longitud n tenemos un coste operacional lineal de $O(n)$

Por lo que nuestro algoritmo tiene una complejidad total de $O(n*n!)$. Esto hace que no sea escalable ya que para valores muy grandes de n el factorial crece de forma exponencial haciendo que el computo de todas las posibles iteraciones aumente muy rápido.

#6. Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

Al tratarse de un problema de tipo NP-hard, no obtendremos una solución exacta del problema. Nuestro problema inicial requiere mucho tiempo de exploración dada la enorme cantidad de posibilidades, por esto un problema de tipo metaheurístico es ideal, ya que explorará nuestro espacio muestral grande pero con una cierta orientación. Lo que hay que tener en cuenta es que no encontraremos la solución óptima pero si una solución de calidad que resuelva el problema.



In [34]:
import random
import math
import copy

'''
Esta función toma dos valores random del conjunto de tomas y cambia su posición, esto lo hace con el objetivo de poder escapar a mínimos locales
que pueden dar una solución que sea buena pero no la mejor.
'''
def generar_vecino(solucion):
    # Hacemos una copia para no modificar la original
    vecino = copy.deepcopy(solucion)

    # Elegimos dos índices aleatorios distintos dentro de la solución
    i, j = random.sample(range(len(vecino)), 2)

    # Intercambiamos los valores
    vecino[i], vecino[j] = vecino[j], vecino[i]

    return vecino


def recocido_simulado(solucion_inicial, temperatura_inicial=1000, tasa_enfriamiento=0.995, iteraciones_max=10000):
  # Inicialización de valores: solución actual, coste actual (usa la función de coste total anterior), temperatura inicial, mejor solución (irá cambiando a lo largo del procesado del problema) y mejor coste
    solucion_actual = list(solucion_inicial)
    coste_actual = coste_total(solucion_actual)
    temperatura = temperatura_inicial

    # Estos dos valores de devuelven al final de la ejecución
    mejor_solucion = list(solucion_actual)
    mejor_coste = coste_actual

    for i in range(iteraciones_max):

        # Generamos un vecino (cambio aleatorio)
        vecino = generar_vecino(solucion_actual)
        coste_vecino = coste_total(vecino)

        # Calculamos la diferencia de coste
        delta = coste_vecino - coste_actual

        # Criterio de Aceptación: si mejora (delta < 0) o si se cumple la condición de la temperatura
        if delta < 0 or random.random() < math.exp(-delta / temperatura):
            solucion_actual = vecino
            coste_actual = coste_vecino

            if coste_actual < mejor_coste: # en caso de tener menor coste que la mejor solución hasta el momento
                mejor_coste = coste_actual
                mejor_solucion = list(solucion_actual)
                print(f"Mejora en iteración {i}: Coste {mejor_coste}")

        # Bajamos la temperatura multiplicando por la tasa de enfriamiento
        temperatura *= tasa_enfriamiento

    return mejor_solucion, mejor_coste


# Generamos una solución inicial aleatoria con las 30 tomas
tomas = 30
solucion_aleatoria = list(range(1, tomas + 1))
random.shuffle(solucion_aleatoria)

print(f"Coste Inicial (Aleatorio): {coste_total(solucion_aleatoria)}")

solucion_optima, coste_optimo = recocido_simulado(
    solucion_aleatoria,
    temperatura_inicial=1000,
    tasa_enfriamiento=0.995,
    iteraciones_max=100000
)

print("\n--- RESULTADO FINAL ---")
print(f"Mejor Coste: {coste_optimo}")
planificacion = {
    'dia_1': solucion_optima[0:6],
    'dia_2': solucion_optima[6:12],
    'dia_3': solucion_optima[12:18],
    'dia_4': solucion_optima[18:24],
    'dia_5': solucion_optima[24:30]
}

print(f"Mejor Planificación: {planificacion}")

Coste Inicial (Aleatorio): 37
Mejora en iteración 36: Coste 35
Mejora en iteración 37: Coste 34
Mejora en iteración 1084: Coste 33
Mejora en iteración 1091: Coste 32
Mejora en iteración 1690: Coste 31
Mejora en iteración 1763: Coste 30
Mejora en iteración 1979: Coste 29
Mejora en iteración 2279: Coste 28
Mejora en iteración 3229: Coste 27

--- RESULTADO FINAL ---
Mejor Coste: 27
Mejor Planificación: {'dia_1': [17, 24, 14, 18, 19, 21], 'dia_2': [28, 9, 25, 7, 16, 6], 'dia_3': [27, 30, 23, 13, 20, 2], 'dia_4': [10, 1, 12, 29, 8, 26], 'dia_5': [4, 22, 15, 3, 5, 11]}


#7. Calcula la complejidad del algoritmo

Para el cálculo de la complejidad de nuestro *algoritmo de recocido simulado* debemos considerar:
1. *coste_total*: al evaluar el coste de una solución de longitud n, la función recorre todas las tomas una vez en bloques de 6, por lo que su complejidad es lineal, $O(n)$.
2. *generación de vecinos*: crear un vecino consiste únicamente en intercambiar dos posiciones de la permutación, lo cual es una operación constante, $O(1)$.

Dado que en cada iteración del algoritmo se genera un vecino en $O(1)$ y se evalúa su coste en $O(n)$, el coste por iteración es $O(n)$. Además, el número total de iteraciones es un parámetro fijo elegido por nosotros y no depende del tamaño del problema. Por tanto, la complejidad total del algoritmo es:
$O(n)$.

Esto hace que el algoritmo sea escalable, ya que su coste crece de forma lineal con el número de tomas, a diferencia del enfoque por fuerza bruta cuyo crecimiento es factorial.

#8. Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

#9. Aplica el algoritmo al juego de datos generado

Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

Respuesta

Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Respuesta